In [75]:
# Carica il file csv e converto colonna dei prezzi in int
import pandas as pd
import re

df=pd.read_csv("cosenza_dati_booking.csv")
df['prezzo'] = df['prezzo'].apply(lambda x: int(re.search(r'\d+', x).group()))
df['prezzo_iniziale'] = df['prezzo_iniziale'].apply(lambda x: int(re.search(r'\d+', x).group()))
try: df['credito_guadagnato'] = df['credito_guadagnato'].str.extract(r'(\d+)').astype(float)
except: pass
len(df[df['evidenza']==True])


4

In [76]:
# Crea delle colonne con valori True/False in base alle offerte
df['booking_paga'] = df['offerte'].str.contains('Booking.com paga')
df['offerta_tempo_limitato'] = df['offerte'].str.contains('tempo limitato')
df['offerta_solo_mobile'] = df['offerte'].str.contains('mobile')
df['offerta_super_segreta'] = df['offerte'].str.contains('SuperSegreta')
df['offerta_inizio_2024'] = df['offerte'].str.contains('Offerta Inizio 2024')
# Inserisce False nei campi nulli
df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']] = df[['booking_paga','offerta_tempo_limitato','offerta_solo_mobile','offerta_super_segreta','offerta_inizio_2024']].fillna(False)

In [77]:
# Visualizzo OS in base a username
df_user = df[df['username']=='pantilaura56']
lst = df_user['TIMESTAMP'].unique()
os = []
for i in lst:
    df3 = df_user[df_user['TIMESTAMP']==i]
    os.append(df3['os'].unique()[0])
os

['Android', 'Windows']

In [78]:
# Visualizza lista ricerche per orario
df['TIMESTAMP'].unique()

array(['03-06-2024---19:11', '03-06-2024---19:13'], dtype=object)

In [79]:
# Divide il dataframe in base a orari ricerche per fare poi il merge
orario1 = df["TIMESTAMP"].unique()[0]
orario2 = df["TIMESTAMP"].unique()[1]

df1 = df[df["TIMESTAMP"]==orario1]
df2 = df[df["TIMESTAMP"]==orario2]

# Eliminazione duplicati
df1 = df1.drop_duplicates()
df2 = df2.drop_duplicates()
df_list = df.columns.tolist()
df_list

['nome_hotel',
 'prezzo_iniziale',
 'prezzo',
 'credito_guadagnato',
 'stanza',
 'città',
 'data',
 'punteggio',
 'stelle',
 'stelle_booking',
 'partner_preferiti',
 'novita',
 'evidenza',
 'numero_recensioni',
 'distanza_centro',
 'genius',
 'offerte',
 'colazione_inclusa',
 'senza_pagamento_anticipato',
 'cancellazione_gratuita',
 'os',
 'username',
 'user_agent_type',
 'TIMESTAMP',
 'booking_paga',
 'offerta_tempo_limitato',
 'offerta_solo_mobile',
 'offerta_super_segreta',
 'offerta_inizio_2024']

In [83]:
# MERGE TRA DATAFRAME
'''
Se le ricerche sono dello stesso tipo (Mobile o Desktop) il merge avviene anche sulla colonna stanza
Vengono inoltre compattate eventuali colonne uguali e prese in considerazione le righe in cui vi è una differenza di prezzo
'''
df_list = df.columns.tolist()
df_list =  df_list[3:4] +df_list[5:6] +df_list[7:]
if df1['user_agent_type'].unique()[0]==df2['user_agent_type'].unique()[0]:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','stanza','data','prezzo_iniziale'], how='inner')
else:
    merged_df = pd.merge(df1, df2, on=['nome_hotel','data','prezzo_iniziale'], how='inner')
merged_df = merged_df[merged_df["prezzo_x"]!=merged_df["prezzo_y"]]
for col in df_list:
    if merged_df[col+"_x"].equals(merged_df[col+"_y"]):
        merged_df[col] = merged_df[col+"_x"]
        merged_df.drop(columns=[col+"_x",col+"_y"],inplace=True)
# Elimino hotel con stesso nome
merged_df = merged_df.drop_duplicates(subset="nome_hotel",keep= False)      
df_list  


['credito_guadagnato',
 'città',
 'punteggio',
 'stelle',
 'stelle_booking',
 'partner_preferiti',
 'novita',
 'evidenza',
 'numero_recensioni',
 'distanza_centro',
 'genius',
 'offerte',
 'colazione_inclusa',
 'senza_pagamento_anticipato',
 'cancellazione_gratuita',
 'os',
 'username',
 'user_agent_type',
 'TIMESTAMP',
 'booking_paga',
 'offerta_tempo_limitato',
 'offerta_solo_mobile',
 'offerta_super_segreta',
 'offerta_inizio_2024']

In [84]:
# Per visualizzare tutte le righe con differenze di prezzo
#pd.set_option('display.max_rows', None)
merged_df[["nome_hotel","prezzo_x","prezzo_y","offerte_x","offerte_y"]]
len(merged_df)


15

In [49]:
#aggiungo colonna con differenza prezzi
pd.set_option('display.max_rows', None)
merged_df["differenza_prezzo"] = (merged_df["prezzo_x"]-merged_df["prezzo_y"])
merged_df = merged_df.sort_values(by='differenza_prezzo',ascending= False)
merged_df = merged_df [merged_df['prezzo_iniziale_x']!= merged_df['prezzo_iniziale_y']]
merged_df[['nome_hotel','prezzo_x','prezzo_y','prezzo_iniziale_x','prezzo_iniziale_y']]

,nome_hotel,prezzo_x,prezzo_y,prezzo_iniziale_x,prezzo_iniziale_y
355,Crossing Manzoni,381,336,423,373
390,Magna Pars l'Hotel à Parfum,491,463,491,589
159,The Corner Duomo Hotel,326,300,326,300
202,Hotel Stradivari,129,105,129,128
291,Hotel Garda,184,163,204,163
126,SCF Collection - Duomo,367,350,477,430
585,Hotel La Caravella,93,81,93,89
135,Montenapoleone Suites,243,231,269,283
367,Hotel Berlino,143,132,158,132
347,Keb Hotel Vitruvio,131,121,199,134


In [ ]:
# Visualizza dove compare una specifica offerta in almeno una delle ricerche
df_mobile = merged_df[(merged_df['booking_paga_x']) ^ (merged_df['booking_paga_y'])]
len(df_mobile)

In [ ]:
# Elimino righe in cui il mobile non ha la stanza
if merged_df['user_agent_type_x'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_x'].dropna()
elif merged_df['user_agent_type_y'].unique()[0] == 'mobile':
    df_senza_stanza = merged_df['stanza_y'].dropna()
len(df_senza_stanza)